# Experiment 7 - CoT

In [ ]:
! tar czf Experiments.tar *

In [ ]:
! nvidia-smi

In [2]:
import pickle
import random

In [3]:
import sys
sys.path.append("../Self-evaluation")

from self_evaluation import play, LLMAgentSelfEvaluate

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Notebook variables

In [4]:
exp_number = 7

seeds = range(1,11)
n_episodes = 1
max_steps = 100
n_think_indices = [4,10]
models = ["0", "e1", "r4", "r10"]

levels = {"coin_collector": 14, "cooking": 9, "treasure_hunter": 27}

In [5]:
with open(f"./games/max_scores_simple.pickle", "rb") as f:
    max_scores_simple = pickle.load(f)
max_scores_simple = max_scores_simple[:len(seeds)]

with open(f"./games/max_scores_cooking.pickle", "rb") as f:
    max_scores_cooking = pickle.load(f)

# Game generation

In [ ]:
import subprocess

for game in levels.keys():
    if game == "cooking":
        continue # we generate cooking in another way
    for level in range(1, levels[game]+1):
        for seed in seeds:
            subprocess.run(["tw-make", f"tw-{game}", "--level", f"{level}", "--seed", f"{seed}", "--output", f"games/{game}/level{level}-seed{seed}.z8"]) 

In [ ]:
# redo coin collector level 8 with more seeds
for seed in seeds:
    subprocess.run(["tw-make", "tw-coin_collector", "--level", "8", "--seed", f"{seed}", "--output", f"games/coin_collector/level8-seed{seed}.z8"])

# play immediately
for n in n_think_indices:
    results_all_seeds = []
    for seed in seeds:
        results = play(LLMAgentSelfEvaluate(selfeval_turns=n, verbose=False),
                f"games/coin_collector/level8-seed{seed}.z8", max_steps=max_steps, n_episodes=n_episodes)
        results_all_seeds.append(results[0]) # only one episode
    with open(f"./Testing 4/{n}think_coin_collector_level8", "wb") as f:
        pickle.dump(results_all_seeds, f)
        print("Data pickled.")
        f.close()

In [ ]:
# treasure hunter levels 16-30

import subprocess

for game in levels.keys():
    if game == "cooking" or game == "coin_collector":
        continue # we generate in another way
    for level in range(16, 30):
        for seed in seeds:
            subprocess.run(["tw-make", f"tw-{game}", "--level", f"{level}", "--seed", f"{seed}", "--output", f"games/{game}/level{level}-seed{seed}.z8"]) 

In [ ]:
!tw-make tw-treasure_hunter --seed 11 --level 20 --output games/treasure_hunter/level20-seed3.z8

In [ ]:
!tw-make tw-treasure_hunter --seed 13 --level 9 --output games/treasure_hunter/level9-seed8.z8
!tw-make tw-treasure_hunter --seed 15 --level 9 --output games/treasure_hunter/level9-seed9.z8
!tw-make tw-treasure_hunter --seed 13 --level 10 --output games/treasure_hunter/level10-seed8.z8
!tw-make tw-treasure_hunter --seed 15 --level 10 --output games/treasure_hunter/level10-seed9.z8

In [ ]:
# coin collector levels 16-20 seed 10

import subprocess
for game in levels.keys():
    if game == "cooking" or game == "treasure_hunter":
        continue
    for level in range(16, 21):
        for seed in [10]:
            subprocess.run(["tw-make", f"tw-{game}", "--level", f"{level}", "--seed", f"{seed}", "--output", f"games/{game}/level{level}-seed{seed}.z8"])

#### Cooking

In [ ]:
!tw-make tw-cooking --seed 1 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed1.z8
!tw-make tw-cooking --seed 2 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed2.z8
!tw-make tw-cooking --seed 3 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed3.z8
!tw-make tw-cooking --seed 4 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed4.z8
!tw-make tw-cooking --seed 5 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed5.z8
!tw-make tw-cooking --seed 6 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed6.z8
!tw-make tw-cooking --seed 7 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed7.z8
!tw-make tw-cooking --seed 8 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed8.z8
!tw-make tw-cooking --seed 9 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed9.z8
!tw-make tw-cooking --seed 10 --go 1 --take 1 --recipe 1 --cut --cook --output games/cooking/level1-seed10.z8

!tw-make tw-cooking --seed 1 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed1.z8
!tw-make tw-cooking --seed 2 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed2.z8
!tw-make tw-cooking --seed 3 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed3.z8
!tw-make tw-cooking --seed 4 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed4.z8
!tw-make tw-cooking --seed 5 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed5.z8
!tw-make tw-cooking --seed 6 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed6.z8
!tw-make tw-cooking --seed 7 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed7.z8
!tw-make tw-cooking --seed 8 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed8.z8
!tw-make tw-cooking --seed 9 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed9.z8
!tw-make tw-cooking --seed 10 --go 1 --take 2 --recipe 2 --cut --cook --output games/cooking/level2-seed10.z8

!tw-make tw-cooking --seed 1 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed1.z8
!tw-make tw-cooking --seed 2 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed2.z8
!tw-make tw-cooking --seed 3 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed3.z8
!tw-make tw-cooking --seed 4 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed4.z8
!tw-make tw-cooking --seed 5 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed5.z8
!tw-make tw-cooking --seed 6 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed6.z8
!tw-make tw-cooking --seed 7 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed7.z8
!tw-make tw-cooking --seed 8 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed8.z8
!tw-make tw-cooking --seed 9 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed9.z8
!tw-make tw-cooking --seed 10 --go 1 --take 3 --recipe 3 --cut --cook --output games/cooking/level3-seed10.z8

!tw-make tw-cooking --seed 1 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed1.z8
!tw-make tw-cooking --seed 2 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed2.z8
!tw-make tw-cooking --seed 3 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed3.z8
!tw-make tw-cooking --seed 4 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed4.z8
!tw-make tw-cooking --seed 5 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed5.z8
!tw-make tw-cooking --seed 6 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed6.z8
!tw-make tw-cooking --seed 7 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed7.z8
!tw-make tw-cooking --seed 8 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed8.z8
!tw-make tw-cooking --seed 9 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed9.z8
!tw-make tw-cooking --seed 10 --go 6 --take 1 --recipe 1 --cut --cook --output games/cooking/level4-seed10.z8

!tw-make tw-cooking --seed 1 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed1.z8
!tw-make tw-cooking --seed 2 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed2.z8
!tw-make tw-cooking --seed 3 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed3.z8
!tw-make tw-cooking --seed 4 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed4.z8
!tw-make tw-cooking --seed 5 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed5.z8
!tw-make tw-cooking --seed 6 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed6.z8
!tw-make tw-cooking --seed 7 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed7.z8
!tw-make tw-cooking --seed 8 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed8.z8
!tw-make tw-cooking --seed 9 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed9.z8
!tw-make tw-cooking --seed 10 --go 6 --take 2 --recipe 2 --cut --cook --output games/cooking/level5-seed10.z8

In [ ]:
!tw-make tw-cooking --seed 1 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed1.z8
!tw-make tw-cooking --seed 2 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed2.z8
!tw-make tw-cooking --seed 3 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed3.z8
!tw-make tw-cooking --seed 4 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed4.z8
!tw-make tw-cooking --seed 5 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed5.z8
!tw-make tw-cooking --seed 6 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed6.z8
!tw-make tw-cooking --seed 7 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed7.z8
!tw-make tw-cooking --seed 8 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed8.z8
!tw-make tw-cooking --seed 9 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed9.z8
!tw-make tw-cooking --seed 10 --go 6 --take 3 --recipe 3 --cut --cook --output games/cooking/level6-seed10.z8

In [ ]:
!tw-make tw-cooking --seed 1 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed1.z8
!tw-make tw-cooking --seed 2 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed2.z8
!tw-make tw-cooking --seed 3 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed3.z8
!tw-make tw-cooking --seed 4 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed4.z8
!tw-make tw-cooking --seed 5 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed5.z8
!tw-make tw-cooking --seed 6 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed6.z8
!tw-make tw-cooking --seed 7 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed7.z8
!tw-make tw-cooking --seed 8 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed8.z8
!tw-make tw-cooking --seed 9 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed9.z8
!tw-make tw-cooking --seed 10 --go 6 --take 3 --recipe 3 --cut --cook --open --output games/cooking/level7-seed10.z8

In [ ]:
!tw-make tw-cooking --seed 1 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed1.z8
!tw-make tw-cooking --seed 2 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed2.z8
!tw-make tw-cooking --seed 3 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed3.z8
!tw-make tw-cooking --seed 4 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed4.z8
!tw-make tw-cooking --seed 5 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed5.z8
!tw-make tw-cooking --seed 6 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed6.z8
!tw-make tw-cooking --seed 7 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed7.z8
!tw-make tw-cooking --seed 8 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed8.z8
!tw-make tw-cooking --seed 9 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed9.z8
!tw-make tw-cooking --seed 10 --go 6 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level8-seed10.z8

In [ ]:
!tw-make tw-cooking --seed 1 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed1.z8
!tw-make tw-cooking --seed 2 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed2.z8
!tw-make tw-cooking --seed 3 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed3.z8
!tw-make tw-cooking --seed 4 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed4.z8
!tw-make tw-cooking --seed 5 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed5.z8
!tw-make tw-cooking --seed 6 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed6.z8
!tw-make tw-cooking --seed 7 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed7.z8
!tw-make tw-cooking --seed 8 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed8.z8
!tw-make tw-cooking --seed 9 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed9.z8
!tw-make tw-cooking --seed 10 --go 9 --take 3 --recipe 3 --cut --cook --open --drop --output games/cooking/level9-seed10.z8

# Game running

In [6]:
n_episodes = 1
max_steps = 100

Iter:
- 0think we recycled from testing 5
- first 4 and 10think
- then e1think last

In [ ]:
# 4/10

for n in n_think_indices:
    for game in levels.keys():
        for level in range(1, levels[game]+1):
            results_all_seeds = []
            for seed in seeds:
                results = play(LLMAgentSelfEvaluate(selfeval_turns=n,
                                                    reads_own_reasoning=True,
                                                    random_selfeval=True,
                                                    prompt_version="CoT",
                                                    log=f"./Experiment {exp_number}/logs/{game}/level{level}-seed{seed}.log"
                                                    ),
                               f"games/{game}/level{level}-seed{seed}.z8",
                               max_steps=max_steps,
                               n_episodes=n_episodes)
                results_all_seeds.append(results[0])
            with open(f"./Experiment {exp_number}/r{n}think_{game}_level{level}.pickle", "wb") as f:
                pickle.dump(results_all_seeds, f)
                print("Data pickled.")

In [ ]:
# e1think

for game in levels.keys():
    
    for level in range(1, levels[game]+1):
        if (game != "treasure_hunter"):
            continue # skip runs we've already done
        results_all_seeds = []
        for seed in seeds:
            results = play(LLMAgentSelfEvaluate(selfeval_turns=1,
                                              reads_own_reasoning=False,
                                              random_selfeval=False,
                                              prompt_version="CoT",
                                              log=f"./Experiment {exp_number}/logs/{game}/level{level}-seed{seed}.log"
                                              ),
                          f"games/{game}/level{level}-seed{seed}.z8",
                          max_steps=max_steps,
                          n_episodes=n_episodes)
            results_all_seeds.append(results[0])
        with open(f"./Experiment {exp_number}/e1think_{game}_level{level}.pickle", "wb") as f:
            pickle.dump(results_all_seeds, f)
            print("Data pickled.")

games/treasure_hunter/level1-seed1.z8 

In [ ]:
play(LLMAgentSelfEvaluate(selfeval_turns=1,
                        reads_own_reasoning=False,
                        random_selfeval=False,
                        prompt_version="CoT",
                        verbose=True,
                        log=f"./Experiment {exp_number}/logs/coin_collector/level5-seed10.log"
                        ),
    f"games/coin_collector/level5-seed10.z8",
    max_steps=max_steps,
    n_episodes=n_episodes)

# Data aggregation for visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy.stats import bootstrap

In [ ]:
final_arrays_times = {"coin_collector": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])},
                "treasure_hunter": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])},
                "cooking": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])}}
final_arrays_time = {"coin_collector": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])},
                    "treasure_hunter": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])},
                    "cooking": {"0": ([],[]), "e1": ([],[]), "r4":([],[]), "r10": ([],[])}}

for n in models:
    for game in levels.keys():
        for level in range(1, levels[game]+1):
            results_all_seeds = []
            with open(f"./Experiment {exp_number}/{n}think_{game}_level{level}.pickle", "rb") as f:
                results_all_seeds = pickle.load(f)
            
            final_scores = [run[-1][1] for run in results_all_seeds] # score
            if game == "cooking":
                final_scores = np.array(final_scores) / max_scores_cooking[level - 1]
            avg_final_score = np.mean(final_scores)
            bootstrap_score = bootstrap(data=(final_scores,), 
                                statistic=np.mean,
                                method="basic",
                                n_resamples=1000,
                                confidence_level=0.9)
            final_arrays_times[game][n][0].append(avg_final_score)
            final_arrays_times[game][n][1].append((bootstrap_score.confidence_interval.high, bootstrap_score.confidence_interval.low))

            final_times = [run[-1][2] for run in results_all_seeds] # time
            avg_final_time = np.mean(final_times)
            bootstrap_time = bootstrap(data=(final_times,), 
                                statistic=np.mean,
                                method="basic",
                                n_resamples=1000,
                                confidence_level=0.9)
            final_arrays_time[game][n][0].append(avg_final_time)
            final_arrays_time[game][n][1].append((bootstrap_time.confidence_interval.high, bootstrap_time.confidence_interval.low))

# Visualization

In [ ]:
figsize=(30,12)

In [ ]:
custom_palette = ['#c44e52', '#55a868', '#64b5cd', '#ccb974']

In [ ]:
fig, axes = plt.subplots(3,1, figsize=figsize)
prettified_names = {"coin_collector": "Coin Collector", "cooking": "Cooking", "treasure_hunter": "Treasure Hunter"}

for i, game in enumerate(final_arrays_times.keys()):
    ax = axes[i]
    x = list(range(1, levels[game]+1))
    for j, model in enumerate(models):
        y, y_ci = final_arrays_times[game][0]

        ax.fill_between(x,
                        np.clip([k[0] for k in y_ci], 0, 1),
                        np.clip([k[1] for k in y_ci], 0, 1),
                        alpha=0.1,
                        color=custom_palette[j])

        ax.plot(x, y, label="0-think", marker=".", linestyle="-", color=custom_palette[j], linewidth=4-(i/2))
    
    ax.set_xlabel('level')
    ax.set_ylabel('normalized average final score')
    ax.set_title(f"Average final score in each level of {prettified_names[game]}")
    ax.legend()

    ax.set_xticks(x)
    ax.grid()

plt.show()

In [ ]:
fig, axes = plt.subplots(3,1, figsize=figsize)
prettified_names = {"coin_collector": "Coin Collector", "cooking": "Cooking", "treasure_hunter": "Treasure Hunter"}

for i, game in enumerate(final_arrays_times.keys()):
    ax = axes[i]
    x = list(range(1, levels[game]+1))
    for j, model in enumerate(models):
        y, y_ci = final_arrays_times[game][0]

        ax.fill_between(x,
                        np.clip([k[0] for k in y_ci], 0, a_max=None),
                        np.clip([k[1] for k in y_ci], 0, a_max=None),
                        alpha=0.1,
                        color=custom_palette[j])

        ax.plot(x, y, label="0-think", marker=".", linestyle="-", color=custom_palette[j], linewidth=4-(i/2))
    
    ax.set_xlabel('level')
    ax.set_ylabel('average execution time [s]')
    ax.set_title(f"Average execution time in each level of {prettified_names[game]}")
    ax.legend()

    ax.set_xticks(x)
    ax.grid()

plt.show()